# <p style="text-align: center;"> <span style="color:yellowgreen"> Running the MM </span></p>

Finally, we will program the real MM.  
Once your simulation code is running perfectly, we will test in the real MM 🦆

## Instructions for first run 

<span style="color:coral">NOTE: These instructions only need to be followed once (or a few times).</span>   
They take sometime.  
What we need to run them:

- A new python library that has not been installed previously
- A new lab computer - <span style="color:red">Only use the class computers to connect to the real MMs. DO NOT USE PERSONAL COMPUTERS.</span>
- Each lab group will be assigned a computer and same MM. 
- You will always use the same computer and MM unless there is a major error in them.  
They will be part of your group too 😊 

First, open the `Dockerfile` inside the your main repo directory (Not the ones inside lab1-recipe)  
Change line 2 to match exactly the name of your repo.  
For example, if my repo name is `ee483-group-A`, line 2 will be `ARG REPO_NAME="ee483-group-A"`


### Instructions


#### A. Compiling the docker image

Before we start, make sure you have your repository cloned on the lab computer and updated with the Labs folder. This is done the same way you setup your personal computer.

1. In the HostOS terminal, go to the `Labs` directory using the `cd` command.  
Double check that you are in the `Labs` folder with the `pwd` command. 
It should return: `some_path_in_the_class_computer/Labs`

2. Open the Dockerfile and Edit line 4 to put your names in

3. Compile your docker image locally for the MM with the following command
	- You may need to remove the “- - pull” from the above command if there is no preexisting image.  
	- The arm64v8 is the Jetson Nano architecture  

In [ ]:
dts devel build -a arm64v8 -f --pull 

<br><img src="../assets/lab1-images/MM-real-build.png" style="width: 30%; height: auto;"></br>

4. Find the docker image generated when the command above finishes without errors. Execute the following command in hostOS terminal to find the image name.
	- If there are errors, talk with the instructor or the TA.  
	- Your compiled image you want is usually the most recently created one.

In [ ]:
docker images 

<br><img src="../assets/lab1-images/MM-images.png" style="width: 75%; height: auto;"></br>

5. Save the image in .tar file with the command below
	- Choose your \<filename\>, for example `mmcode`
	- Parameters \<repo\>:\<tag\> are the repository name and tag that show up after running docker images (the image you want to send to the MM).
	- This might take sometime and does not produce any information after running.

In [ ]:
docker save -o ~/<filename>.tar <repo>:<tag> 

<br><img src="../assets/lab1-images/MM-save-image.png" style="width: 75%; height: auto;"></br>

#### B. Sending the image to the MM

1. Connect computer to the classroom network.  
   - The network has no internet access but we can communicate with the MMs.

2. Turn on your MM by pressing the power button on the bottom of the chasis.
<br><img src="../assets/lab1-images/MM-power.png" style="width: 20%; height: auto;"></br>

3. Check if you MM is connected to the classroom network by running the code below.
   - You should see the name of your MM pop up under hostname once your duckiebot is done booting

In [ ]:
dts fleet discover


4. Sending the docker image saved in part A. to an MM using the `scp` command
	- scp works as follows `scp path_file_to_transfer destination_to_send_the_file`
	- Use the command below to transfer the image.  
	Replace <filename> with filename created in previous step  
	Replace <MMname> with the name of your group MM. Make sure it's the correct one, e.g., ee483mm02 if your group has ee483mm02.  
	The password is quackquack
	- We are transfering the image you have saved in A to the home folder in the MM 


In [ ]:
scp ~:/<filename>.tar duckie@<MMname>.local:~/ 

5. The command above might take a few minutes

#### C. Loading the docker image in the MM

1. Accessing the MM using `ssh` (Secure Shell)
	- Replace \<MMname> with the name of the MM   
	- Answer yes to saving ssh fingerprint if this message appears
	- The password is quackquack
	- The ssh command will access a shell in the MM, i.e., once you run the command, we will be in the MM OS using the terminal in the hostOS

In [ ]:
ssh duckie@<MMname>.local 

2. Check if the .tar file got transfer correctly to the MM with the `ls` command
	- You should see that the .tar file is in the home directory of the MM.

In [ ]:
ls

3. Loading the docker image in the duckie  
	- This might take a while ~5-10 mins

In [ ]:
docker load -i <filename>.tar  

4. Delete the .tar file with command `rm` in the MM

In [ ]:
rm <filename>.tar

5. Exit the ssh shell with command `exit`

In [ ]:
exit

6. Delete the .tar file in the hostOS

In [ ]:
rm ~/<filename>.tar

#### D. Testing the image you just loaded in the MM

1. Make sure you are in the `Labs` directory and **NOT** in `lab1`.  
You might need to cd to `Labs`, recall that `cd ..` goes back one directory.  
To make sure you are in the correct dir, check with command `pwd`

2. Running the dts devel run command
	- The -H flag tells that we are running the docker image in the MM
	- The -s copies your repo to the /code folder on the robot so that it can be mounted by the -M flag. This means the code on the robot will resync with your code on your computer so it will update any changes you have made to your code.   
	- The -s will automatically copy the packages you developed in the `Lab1-open/lab1/ex_workspace/src`.   
	You don't have to worry to manually copying it. The same will be set for the other labs.
	- It is unlikely that you will have to rebuild during this lab.
	- It might take sometime to run the image.
	- The --cmd will start a shell in the container you are running - the MM container.  
	- It initiates a terminal exactly the same as the `LXTerminal`. 
		- For example, you can run catkin build and it should build the packages in the container (give it a try).
	- The -f forces when your git repo is not updated.

In [ ]:
dts devel run -H <MMname>.local -s -M --cmd bash -f 

#### E. Common errors
1. If you get a docker conflict saying container is already running, there are two options 

	- Attach to the running container.

		`dts devel run attach`

	- Delete the running container. Do this if you have updates 

		- Connect to shell: ssh duckie@\<MMname\>.local 

		- List docker images: docker ps 

		- Delete the container: docker rm -f \<container_ID\> 

		- Exit the shell: exit 

End of first run instruction

## Usefull MM tools

### Manually driving

1. In a hostOS terminal run the following command
	- More details in https://docs.duckietown.com/daffy/opmanual-duckiebot/operations/make_it_move/index.html 

In [ ]:
dts duckiebot keyboard_control <MMname>

### Observing the camera
1. In a hostOS terminal run the following commands
	- After starting the start_gui_tools, you can also run the `rqt_graph` instead of the `rqt_image_view`.
	- More details in https://docs.duckietown.com/daffy/opmanual-duckiebot/operations/make_it_see/index.html

In [ ]:
dts start_gui_tools <MMname>
rqt_image_view

## Testing your code in the MM

1. Ensure you are connected to the classroom network
2. Check if you MM is connected to the classroom network by running the code below.
   - You should see the name of your MM pop up under hostname


In [ ]:
dts fleet discover

3. Run the `dts devel run` command

In [ ]:
dts devel run -H <MMname>.local -s -M --cmd bash -f 

4. Once the MM shell appear, built and activate the catkin workspace

In [ ]:
catkin_build
source /code/catkin_ws/devel/setup.bash

5. Run the node you want to test or the launch file from your package
	- The package, nodes, and launch files are the ones created 

In [ ]:
rosrun <package_name> <node_name>.py

OR

roslaunch <package_name> <launch_file>.py